In [1]:
import cv2
import numpy as np
from keras.models import load_model
from deepface import DeepFace
import time

# Modeli yükle
model = load_model("model_dropout.h5")

# Yüz tespiti için Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Kamera başlat
cap = cv2.VideoCapture(0)

# Zaman kontrolü
fps_limit = 5
interval = 1.0 / fps_limit
last_time = time.time()

deepface_interval = 2  # saniyede bir DeepFace çalışsın
last_deepface_time = time.time()

# Önceki isimleri ve duyguları sakla
prev_results = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    current_results = []

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_gray = gray[y:y+h, x:x+w]

        # Duygu tahmini (her frame yapılabilir çünkü hızlı)
        resized = cv2.resize(face_img, (48, 48))  # Gri değil, renkli halini al
        img_array = np.expand_dims(resized, axis=0)
        img_array = img_array / 255.0


        prediction = model.predict(img_array, verbose=0)
        prob_happy = prediction[0][0] * 100
        prob_sad = (1 - prediction[0][0]) * 100
        emotion_label = "Happy" if prediction[0][0] > 0.5 else "Sad"
        emotion_text = f"{emotion_label}: {max(prob_happy, prob_sad):.1f}%"

        # Yüz tanıma (belirli aralıklarla)
        name = "Tespit ediliyor..."
        if time.time() - last_deepface_time >= deepface_interval:
            try:
                result = DeepFace.find(img_path=face_img, db_path="dataset", enforce_detection=False)
                if len(result[0]) > 0:
                    identity_path = result[0].iloc[0]["identity"]
                    name = identity_path.split("\\")[-2]
            except Exception as e:
                print("DeepFace hatası:", e)

        current_results.append(((x, y, w, h), name, emotion_text))

    # DeepFace zamanı yenile
    if time.time() - last_deepface_time >= deepface_interval:
        prev_results = current_results
        last_deepface_time = time.time()

    # Ekrana yaz
    for ((x, y, w, h), name, emotion_text) in prev_results:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 255, 100), 2)
        cv2.putText(frame, f"{name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, emotion_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 200, 0), 2)

    # Göster
    cv2.imshow("Kamera", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [5]:
import cv2
import numpy as np
import time
from deepface import DeepFace
from tensorflow.keras.models import load_model

# Modeli yükle
model = load_model("model_dropout.h5")

# Kamera başlat
cap = cv2.VideoCapture(0)

fps_limit = 5
interval = 1.0 / fps_limit
last_time = time.time()

deepface_interval = 5  # saniyede bir yüz tanıma
last_deepface_time = time.time()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Son tanınan yüzleri tutacağımız liste
recognized_faces = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    recognized_faces = []

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]

        # Duygu analizi
        try:
            emotion_face = cv2.resize(face_img, (48, 48))
            emotion_input = np.expand_dims(emotion_face, axis=0) / 255.0
            prediction = model.predict(emotion_input, verbose=0)
            prob_happy = prediction[0][0] * 100
            prob_sad = (1 - prediction[0][0]) * 100
            emotion_label = "Happy" if prediction[0][0] > 0.5 else "Sad"
            emotion_text = f"{emotion_label}: {max(prob_happy, prob_sad):.1f}%"
        except:
            emotion_text = "Duygu bulunamadı"

        # Yüz tanıma (belli aralıklarla)
        name = "Tanınmıyor"
        if current_time - last_deepface_time >= deepface_interval:
            try:
                result = DeepFace.find(img_path=face_img, db_path="dataset", enforce_detection=False)
                if len(result[0]) > 0:
                    identity_path = result[0].iloc[0]["identity"]
                    name = identity_path.split("\\")[-2]
            except Exception as e:
                print("DeepFace Hatası:", e)

        # Sonuçları çiz
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)
        cv2.putText(frame, emotion_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 0), 2)
        cv2.putText(frame, f"{name}", (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # DeepFace'i sadece belirli aralıklarla çalıştır
    if current_time - last_deepface_time >= deepface_interval:
        last_deepface_time = current_time

    # Kamerayı göster
    cv2.imshow("Kamera", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


25-05-07 19:45:32 - Searching [[[159 159 159]
  [159 159 159]
  [158 158 158]
  ...
  [142 142 148]
  [143 143 149]
  [142 143 147]]

 [[159 159 159]
  [159 159 159]
  [158 158 158]
  ...
  [143 143 149]
  [143 143 149]
  [142 143 147]]

 [[160 159 161]
  [160 160 160]
  [159 159 159]
  ...
  [144 142 147]
  [144 143 147]
  [142 144 148]]

 ...

 [[161 161 161]
  [161 161 161]
  [161 161 161]
  ...
  [144 143 145]
  [144 143 145]
  [144 143 145]]

 [[161 161 161]
  [161 161 161]
  [161 161 161]
  ...
  [143 142 146]
  [143 142 146]
  [143 142 145]]

 [[161 161 161]
  [160 160 160]
  [160 160 160]
  ...
  [143 142 146]
  [143 142 146]
  [143 142 145]]] in 14 length datastore
DeepFace Hatası: The layer sequential_112 has never been called and thus has no defined input.
25-05-07 19:45:37 - Searching [[[158 157 159]
  [158 157 159]
  [158 157 159]
  ...
  [141 141 147]
  [141 141 147]
  [144 144 150]]

 [[158 157 159]
  [158 157 159]
  [157 156 158]
  ...
  [141 141 147]
  [141 141 147]
  

In [9]:
import cv2
import numpy as np
from keras.models import load_model
from deepface import DeepFace
import time

# Modeli yükle
model = load_model("model_dropout.h5")

# Yüz tespiti
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Kamera
cap = cv2.VideoCapture(0)

# DeepFace işlemleri için zaman kontrolü
deepface_interval = 2
last_deepface_time = 0
recognized_faces = {}

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    current_time = time.time()
    new_recognized = {}

    for i, (x, y, w, h) in enumerate(faces):
        face_img_color = frame[y:y+h, x:x+w]
        face_img_resized = cv2.resize(face_img_color, (48, 48))
        img_array = np.expand_dims(face_img_resized, axis=0) / 255.0

        # Duygu tahmini
        prediction = model.predict(img_array, verbose=0)
        emotion_label = "Happy" if prediction[0][0] > 0.5 else "Sad"
        prob = max(prediction[0][0], 1 - prediction[0][0]) * 100
        emotion_text = f"{emotion_label}: {prob:.1f}%"

        face_id = f"{x}_{y}_{w}_{h}"
        name = recognized_faces.get(face_id, "Tanınıyor...")

        # Her deepface_interval saniyede bir yüz tanıma
        if current_time - last_deepface_time > deepface_interval:
            try:
                result = DeepFace.find(img_path=face_img_color, db_path="dataset", enforce_detection=False)
                if len(result[0]) > 0:
                    identity_path = result[0].iloc[0]["identity"]
                    name = identity_path.split("\\")[-2]
            except Exception as e:
                print("DeepFace hatası:", e)
            new_recognized[face_id] = name

        # Ekrana yaz
        cv2.rectangle(frame, (x, y), (x+w, y+h), (100, 255, 100), 2)
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, emotion_text, (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 200, 0), 2)

    if current_time - last_deepface_time > deepface_interval:
        recognized_faces = new_recognized
        last_deepface_time = current_time

    cv2.imshow("Kamera", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


25-05-07 20:37:52 - Searching [[[169 162 152]
  [168 163 152]
  [164 161 150]
  ...
  [157 152 151]
  [157 152 151]
  [157 152 151]]

 [[169 162 152]
  [168 163 151]
  [163 161 147]
  ...
  [157 152 151]
  [157 152 151]
  [157 152 151]]

 [[169 162 152]
  [168 163 151]
  [159 157 143]
  ...
  [157 152 151]
  [157 152 151]
  [157 152 151]]

 ...

 [[ 14  14  14]
  [ 14  14  14]
  [ 14  14  14]
  ...
  [ 21  21  21]
  [ 21  21  21]
  [ 21  21  21]]

 [[ 14  14  14]
  [ 14  14  14]
  [ 14  14  14]
  ...
  [ 21  21  21]
  [ 21  21  21]
  [ 20  20  20]]

 [[ 14  14  14]
  [ 14  14  14]
  [ 14  14  14]
  ...
  [ 21  21  21]
  [ 20  20  20]
  [ 20  20  20]]] in 14 length datastore
25-05-07 20:37:53 - find function duration 0.4926879405975342 seconds
25-05-07 20:37:54 - Searching [[[161 165 146]
  [162 166 147]
  [162 166 147]
  ...
  [153 156 147]
  [154 157 148]
  [154 157 148]]

 [[163 165 146]
  [164 166 147]
  [164 166 147]
  ...
  [153 156 147]
  [153 156 147]
  [154 156 147]]

 [[164 16

In [2]:
import cv2
import numpy as np
import joblib
from deepface import DeepFace
from keras.models import load_model
import time

# Yüz tanıma için Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# KNN Modelini Yükle
knn_model = joblib.load("face_knn_model.pkl")

# Duygu tahmini için model
emotion_model = load_model("model_dropout.h5")

# Kamera başlat
cap = cv2.VideoCapture(0)

# Zaman kontrolü
fps_limit = 5
interval = 1.0 / fps_limit
last_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]
        
        # 1. Yüz Tanıma (DeepFace ile)
        try:
            # Yüz tanıma için Facenet modelini kullan
            result = DeepFace.represent(img_path=face_img, model_name="Facenet", enforce_detection=False)
            embedding = result[0]['embedding']  # Yüz embedding'ini alıyoruz
            embedding = np.expand_dims(embedding, axis=0)  # Modelin beklediği formatta 2D array

            # 2. KNN model ile yüz tanıma
            prediction = knn_model.predict(embedding)
            name = str(prediction)  # KNN tahminini metin olarak alıyoruz
            cv2.putText(frame, f"Name: {name}", (x, y + h + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 200, 0), 2)
        except Exception as e:
            print(f"Yüz Tanıma Hatası: {e}")

        # 3. Duygu Tahmini (Emotion Modeli)
        try:
            # Duygu analizi için modeli kullan
            resized_face = cv2.resize(face_img, (48, 48))
            resized_face = np.expand_dims(resized_face, axis=0)
            resized_face = resized_face / 255.0  # normalize et
            prediction = emotion_model.predict(resized_face)
            emotion_label = "Sad" if prediction[0][0] > 0.5 else "Happy"
            emotion_text = f"{emotion_label}: {max(prediction[0][0], 1 - prediction[0][0]) * 100:.1f}%"
            cv2.putText(frame, emotion_text, (x, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 200, 0), 2)
        except Exception as e:
            print(f"Duygu Analizi Hatası: {e}")

    # Göster
    cv2.imshow("Kamera", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━